### Web Scraping 

#### In this notebook we can see the code, that I have used to obtain the data from Dia webpage. 

In [34]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm

def get_html(url):
    try:
        data = requests.get(url)
        html = data.text         
        return html
    except:
        print("URL NOT FOUND")

def get_products(url):
    url_html = get_html(url)
    url_html = BeautifulSoup(url_html)
    script_tag = url_html.body.find("script", id="vike_pageContext")
    script_tag = str(script_tag.string)
    d_prod = json.loads(script_tag)
    prods = d_prod["INITIAL_STATE"]["plp_items"][:-1]
    return prods

def flatten_prods(ls_prods):
    ls_prods_flat = []
    # for d_prod in ls_prods:
    for prod in ls_prods:
        d_price = prod["prices"]
        d_prod_flat = prod.copy()
        d_prod_flat.update(d_price)
        del d_prod_flat["prices"]

        ls_prods_flat.append(d_prod_flat)
    return ls_prods_flat

df_dia = pd.read_csv('data/dynamic/dia/Dia-categories-url.csv', sep=',', )
df_dia = df_dia.iloc[:, :4]
df_dia.columns = [x.strip() for x in df_dia.columns]
df_dia

,Cat,Subcat,url,page
0,Charcutería y quesos,"Jamón cocido, lacón, fiambres y mortadela",https://www.dia.es/charcuteria-y-quesos/jamon-...,3
1,Charcutería y quesos,Jamón curado y paleta,https://www.dia.es/charcuteria-y-quesos/jamon-...,2
2,Charcutería y quesos,"Lomo, chorizo, fuet, salchichón",https://www.dia.es/charcuteria-y-quesos/lomo-c...,3
3,Charcutería y quesos,"Queso curado, semicurado y tierno",https://www.dia.es/charcuteria-y-quesos/queso-...,3
4,Charcutería y quesos,Queso fresco,https://www.dia.es/charcuteria-y-quesos/queso-...,1
...,...,...,...,...
172,Bebé,Pañales y toallitas,https://www.dia.es/bebe/panales-y-toallitas/c/...,2
173,Bebé,Cuidado del bebé,https://www.dia.es/bebe/cuidado-del-bebe/c/L2143,2
174,Mascotas,Perros,https://www.dia.es/mascotas/perros/c/L2174,4
175,Mascotas,Gatos,https://www.dia.es/mascotas/gatos/c/L2175,5


In [40]:
ls_dfs = []
for idx, row in tqdm(df_dia.iterrows(), total=df_dia.shape[0]):
    products_tot = []
    for i in range(1, row['page']+1):
        sleep(1)
        url = row['url']
        if i > 1:
            url = url.replace('/c/', f'/pag-{i}/c/')
        products = get_html(url)
        products = get_products(url)
        products = flatten_prods(products)
        products_tot.extend(products)

    df_products = pd.DataFrame(products_tot)
    df_products['id'] = idx
    ls_dfs.append(df_products)

df_products_total = pd.concat(ls_dfs, ignore_index=True)

100%|██████████| 177/177 [15:18<00:00,  5.19s/it] 


In [42]:
df_products_total.to_csv('data/dynamic/dia/Dia-products.csv', index=False)